<a href="https://colab.research.google.com/github/Harjot4026/Classification-with-an-Academic-Success-Dataset/blob/main/Academic_Success1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 8.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import statsmodels.api as sm
from statsmodels.formula.api import ols
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from scipy.stats import t,f,normaltest
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
import datetime
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import chi2_contingency
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import lightgbm as lgb
import numpy as np
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Academic Success/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Academic Success/test.csv')

In [ ]:
df

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,3,1,1,3,9500,1,1,131.0,1,19,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,4,1,1,2,9500,1,1,132.0,1,19,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,76513,1,17,1,9254,1,1,121.0,1,19,...,0,6,8,5,10.600000,0,13.9,-0.3,0.79,Graduate
76514,76514,1,1,6,9254,1,1,125.0,1,1,...,0,6,9,6,13.875000,0,9.4,-0.8,-3.12,Graduate
76515,76515,5,17,1,9085,1,1,138.0,1,37,...,0,5,8,5,11.400000,1,9.4,-0.8,-3.12,Enrolled
76516,76516,1,1,3,9070,1,1,136.0,1,38,...,0,6,0,0,0.000000,0,7.6,2.6,0.32,Dropout


In [ ]:
df['Course_x_Curricular_units_1st_sem_enrolled'] = df['Course']*df['Curricular units 1st sem (enrolled)']
df['Course_x_Curricular_units_2nd_sem_enrolled'] = df['Course']*df['Curricular units 2nd sem (enrolled)']

In [ ]:
df['IR_UR_GDP'] = df['Inflation rate']*df['Unemployment rate']*df['GDP']
df['IR_UR_GDP']

0         13.4532
1         13.4532
2         -4.4712
3         13.4532
4          6.3232
           ...   
76513     -3.2943
76514     23.4624
76515     23.4624
76516      6.3232
76517   -176.2040
Name: IR_UR_GDP, Length: 76518, dtype: float64

In [ ]:
df_test['Course_x_Curricular_units_1st_sem_enrolled'] = df_test['Course']*df_test['Curricular units 1st sem (enrolled)']
df_test['Course_x_Curricular_units_2nd_sem_enrolled'] = df_test['Course']*df_test['Curricular units 2nd sem (enrolled)']

In [ ]:
df_test['IR_UR_GDP'] = df_test['Inflation rate']*df_test['Unemployment rate']*df_test['GDP']
df_test['IR_UR_GDP']

0         -3.2943
1         13.4532
2       -176.2040
3         43.7346
4          6.3232
           ...   
51007   -176.2040
51008     23.4624
51009   -176.2040
51010      6.3232
51011      6.3232
Name: IR_UR_GDP, Length: 51012, dtype: float64

In [ ]:
df.columns

Index(['id', 'Marital status', 'Application mode', 'Application order',
       'Course', 'Daytime/evening attendance', 'Previous qualification',
       'Previous qualification (grade)', 'Nacionality',
       'Mother's qualification', 'Father's qualification',
       'Mother's occupation', 'Father's occupation', 'Admission grade',
       'Displaced', 'Educational special needs', 'Debtor',
       'Tuition fees up to date', 'Gender', 'Scholarship holder',
       'Age at enrollment', 'International',
       'Curricular units 1st sem (credited)',
       'Curricular units 1st sem (enrolled)',
       'Curricular units 1st sem (evaluations)',
       'Curricular units 1st sem (approved)',
       'Curricular units 1st sem (grade)',
       'Curricular units 1st sem (without evaluations)',
       'Curricular units 2nd sem (credited)',
       'Curricular units 2nd sem (enrolled)',
       'Curricular units 2nd sem (evaluations)',
       'Curricular units 2nd sem (approved)',
       'Curricular units

In [ ]:
cols = ['Marital status', 'Application mode','Application order','Course','Curricular units 1st sem (credited)',
       'Curricular units 1st sem (enrolled)',
       'Curricular units 1st sem (evaluations)',
       'Curricular units 1st sem (approved)',
       'Curricular units 1st sem (grade)',
       'Curricular units 1st sem (without evaluations)',
       'Curricular units 2nd sem (credited)',
       'Curricular units 2nd sem (enrolled)',
       'Curricular units 2nd sem (evaluations)',
       'Curricular units 2nd sem (approved)',
       'Curricular units 2nd sem (grade)',
       'Curricular units 2nd sem (without evaluations)', 'Unemployment rate',
       'Inflation rate', 'GDP', 'Previous qualification',
       'Previous qualification (grade)', 'Nacionality',
       "Mother's qualification", "Father's qualification",
       "Mother's occupation", "Father's occupation" , 'Admission grade','Age at enrollment',
        'Course_x_Curricular_units_1st_sem_enrolled','Course_x_Curricular_units_2nd_sem_enrolled']
scaler = StandardScaler()
df[cols] = scaler.fit_transform(df[cols])

In [ ]:
df

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target,Course_x_Curricular_units_1st_sem_enrolled,Course_x_Curricular_units_2nd_sem_enrolled,IR_UR_GDP
0,0,-0.253437,-0.902423,-0.524066,0.131258,1,-0.308308,-0.580138,-0.066801,-1.223274,...,0.718660,0.505317,-0.135127,-0.158418,-0.449110,0.933176,Graduate,0.041892,0.014049,13.4532
1,1,-0.253437,0.056682,-0.524066,0.131258,1,-0.308308,-0.671086,-0.066801,-0.054394,...,-1.445110,-1.735681,-0.135127,-0.158418,-0.449110,0.933176,Dropout,0.041892,0.014049,13.4532
2,2,-0.253437,0.056682,0.289183,0.140130,1,-0.308308,0.420293,-0.066801,-1.093399,...,-1.445110,-1.735681,-0.135127,1.763675,-0.663578,-0.372698,Dropout,0.047992,0.020212,-4.4712
3,3,-0.253437,-0.902423,1.102431,0.276537,1,-0.308308,-0.125396,-0.066801,-0.054394,...,1.079288,0.575895,-0.135127,-0.158418,-0.449110,0.933176,Enrolled,0.745493,1.334575,13.4532
4,4,-0.253437,-0.902423,0.289183,0.276537,1,-0.308308,-0.034448,-0.066801,-0.054394,...,0.718660,0.596330,-0.135127,-1.477502,0.980680,0.178079,Graduate,0.745493,0.724766,6.3232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,76513,-0.253437,0.056682,-0.524066,0.140130,1,-0.308308,-1.034880,-0.066801,-0.054394,...,0.358031,0.175607,-0.135127,0.896848,-1.092515,0.386841,Graduate,0.047992,0.020212,-3.2943
76514,76514,-0.253437,-0.902423,3.542175,0.140130,1,-0.308308,-0.671086,-0.066801,-1.223274,...,0.718660,0.766123,-0.135127,-0.799116,-1.449963,-1.349882,Graduate,0.047992,0.020212,23.4624
76515,76515,8.803180,0.056682,-0.524066,0.046419,1,-0.308308,0.511242,-0.066801,1.114486,...,0.358031,0.319855,2.028889,-0.799116,-1.449963,-1.349882,Enrolled,-0.593776,-0.628048,23.4624
76516,76516,-0.253437,-0.902423,1.102431,0.038102,1,-0.308308,0.329345,-0.066801,1.179424,...,-1.445110,-1.735681,-0.135127,-1.477502,0.980680,0.178079,Dropout,-0.022164,-0.050655,6.3232


In [ ]:
scaler = StandardScaler()
df_test[cols] = scaler.fit_transform(df_test[cols])
df_test

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Course_x_Curricular_units_1st_sem_enrolled,Course_x_Curricular_units_2nd_sem_enrolled,IR_UR_GDP
0,76518,-0.249023,-0.904712,-0.524549,0.270480,1,-0.307212,0.796651,-0.061289,-1.099430,...,-2.118802,-1.469064,-1.766338,-0.136585,0.897514,-1.089794,0.389264,0.747695,1.346001,-3.2943
1,76519,-0.249023,-0.904712,-0.524549,0.120878,1,-0.307212,-0.397591,-0.061289,-1.229438,...,-0.371118,0.712878,0.689656,-0.136585,-0.158656,-0.448201,0.935535,0.031209,0.003877,13.4532
2,76520,-0.249023,-0.904712,-0.524549,0.120878,1,-0.307212,-1.316238,-0.061289,-1.229438,...,1.085285,0.349221,0.234843,-0.136585,1.501040,1.120136,-1.764733,0.031209,0.003877,-176.2040
3,76521,-0.249023,1.677245,-0.524549,0.068917,1,4.123156,-0.213861,-0.061289,-1.229438,...,1.959127,0.349221,0.234843,-0.136585,-0.988505,0.122103,1.597278,-0.004123,1.161763,43.7346
4,76522,-0.249023,1.377018,-0.524549,0.367551,1,-0.307212,-2.051156,-0.061289,-1.229438,...,0.502724,-0.014436,0.174201,4.144503,-1.478870,0.977560,0.180526,0.198941,0.172980,6.3232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51007,127525,-0.249023,-0.904712,0.284739,-5.056386,1,-0.307212,-0.397591,-0.061289,1.175709,...,-2.118802,-1.469064,-1.766338,-0.136585,1.501040,1.120136,-1.764733,-3.555616,-3.612264,-176.2040
51008,127526,2.033667,1.377018,-0.524549,0.052929,1,1.791383,0.070919,-0.061289,-0.059367,...,-0.662399,-1.469064,-1.766338,-0.136585,-0.799903,-1.446235,-1.347257,-0.605098,-0.637631,23.4624
51009,127527,-0.249023,-0.904712,-0.524549,-5.056386,1,-0.307212,-0.489455,-0.061289,-1.229438,...,-2.118802,-1.469064,-1.766338,-0.136585,1.501040,1.120136,-1.764733,-3.555616,-3.612264,-176.2040
51010,127528,-0.249023,-0.904712,1.094027,0.426364,1,-0.307212,-0.030132,-0.061289,-0.059367,...,0.502724,-0.378093,0.598693,-0.136585,-1.478870,0.977560,0.180526,0.238933,0.213299,6.3232


In [ ]:
features = df.columns.drop(['id','Target'])

# X_test = df_test[features]
X = df[features]
y = df['Target']

In [ ]:
X_test = df_test[features]

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
params = {'n_estimators': 8000,
          'num_class': 3,
          'boosting_type': 'gbdt',
          'objective': 'multiclass',
          'metric': 'multi_logloss',
          'verbosity': -1,
          'random_state': 99,
          'reg_alpha': 1.7878527151970849,
          'reg_lambda': 1.391543710164331,
          'colsample_bytree': 0.5,
          'subsample': 0.5,
          'learning_rate': 0.04,
          'max_depth': 20,
          'num_leaves': 70,
          'min_child_samples': 40,
          'min_data_per_groups': 16
         }

In [ ]:
def cross_val_train(X, y, test, params):

    spl=20  # Number of folders

    # Inititate arrays with predictions and oof predictions
    test_preds = np.zeros((len(test),3))
    val_preds = np.zeros((len(X),3))
    val_scores, train_scores = [],[]

    # perform cross-validation split
    cv = KFold(spl, shuffle=True, random_state=42)

    # "for" cycle to train for each fold
    for fold, (train_ind, valid_ind) in enumerate(cv.split(X,y)):

        # divide train and validation data
        X_train = X.iloc[train_ind]
        y_train = y[train_ind]
        X_val = X.iloc[valid_ind]
        y_val = y[valid_ind]

        # Initiate model lightGBM
        model = lgb.LGBMClassifier(**params)

        # fit the model
        model.fit(X_train, y_train,
                    eval_set=[(X_val, y_val)],
                    callbacks=[lgb.early_stopping(stopping_rounds=70), lgb.log_evaluation(100)])

        # predictions on train and validation data
        y_pred_trn = model.predict_proba(X_train)
        y_pred_val = model.predict_proba(X_val)

        # compute accuracy
        train_acc = accuracy_score(y_train, np.argmax(y_pred_trn, axis=1))
        val_acc = accuracy_score(y_val, np.argmax(y_pred_val, axis=1))

        # print partial results for the fold
        print("Fold:",fold, " Train R2:",np.round(train_acc,5), " Val R2:",np.round(val_acc,5))

        # compute test predictions and oof predictions
        test_preds += model.predict_proba(test[features])/spl
        val_preds[valid_ind] = model.predict_proba(X_val)
        val_scores.append(val_acc)
        print("-"*50)

    return val_scores, val_preds, test_preds

In [ ]:
val_scores, val_preds, test_preds = cross_val_train(X, y,df_test, params)


Training until validation scores don't improve for 70 rounds
[100]	valid_0's multi_logloss: 0.442694
[200]	valid_0's multi_logloss: 0.430782
[300]	valid_0's multi_logloss: 0.42956
[400]	valid_0's multi_logloss: 0.429275
Early stopping, best iteration is:
[380]	valid_0's multi_logloss: 0.428946
Fold: 0  Train R2: 0.8766  Val R2: 0.83455
--------------------------------------------------
Training until validation scores don't improve for 70 rounds
[100]	valid_0's multi_logloss: 0.444886
[200]	valid_0's multi_logloss: 0.432536
[300]	valid_0's multi_logloss: 0.431265
Early stopping, best iteration is:
[320]	valid_0's multi_logloss: 0.431125
Fold: 1  Train R2: 0.8708  Val R2: 0.83298
--------------------------------------------------
Training until validation scores don't improve for 70 rounds
[100]	valid_0's multi_logloss: 0.427068
[200]	valid_0's multi_logloss: 0.417333
[300]	valid_0's multi_logloss: 0.417169
Early stopping, best iteration is:
[240]	valid_0's multi_logloss: 0.416967
Fold:

In [ ]:
val_preds_out = np.argmax(val_preds, axis=1)
accuracy_score(y, val_preds_out)

0.8338430173292558

In [ ]:
y_test=np.argmax(test_preds, axis=1)
y_test

array([0, 2, 2, ..., 0, 0, 0])

In [ ]:
y_test = label_encoder.inverse_transform(y_test)

In [ ]:
def print_category_percentages(y_test):
    # Calculate the counts of each category
    category_counts = y_test.value_counts()

    # Calculate the total number of instances
    total_count = len(y_test)

    # Calculate the percentage for each category
    category_percentages = (category_counts / total_count) * 100

    # Print the results
    print("Category Percentages in y_test:")
    for category, percentage in category_percentages.items():
        print(f"{category}: {percentage:.2f}%")

# Example usage:
# Assuming y_test is your test set labels
print_category_percentages(y_test)

Category Percentages in y_test:
Dropout: 30.11%
Enrolled: 17.68%
Graduate: 52.21%


In [ ]:
a = df.to_csv('train.csv',index = False)
b = df_test.to_csv('test.csv',index = False)